In [2]:
import torch.nn as nn
import torch

class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)  # New
        self.register_buffer('mask', torch.triu(torch.ones(
            context_length, context_length), diagonal=1))  # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape  # New batch dimension b
        # For inputs where `num_tokens` exceeds `context_length`, this will result in errors
        # in the mask creation further below.
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs
        # do not exceed `context_length` before reaching this forward method.
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)  # Changed transpose
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights)  # New

        context_vec = attn_weights @ values
        return context_vec

In [3]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, 
                 qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias)
             for _ in range(num_heads)]
        )
    
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [8]:
torch.manual_seed(123)

inputs = torch.tensor(
    [[0.43, 0.15, 0.89],  # Your     (x^1)
     [0.55, 0.87, 0.66],  # journey  (x^2)
     [0.57, 0.85, 0.64],  # starts   (x^3)
     [0.22, 0.58, 0.33],  # with     (x^4)
     [0.77, 0.25, 0.10],  # one      (x^5)
     [0.05, 0.80, 0.55]]  # step     (x^6)
)
batch = torch.stack((inputs, inputs), dim=0) # this is for simulate batch input
context_length = batch.shape[1]
d_in, d_out = 3, 2
num_heads = 10
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads)

context_vecs = mha(batch)

print(context_vecs)
print(context_vecs.shape)


tensor([[[-4.5192e-01,  2.2160e-01,  4.7720e-01,  1.0635e-01,  4.5655e-01,
           2.7288e-01, -5.6836e-01,  5.0635e-01, -4.8215e-01,  4.3357e-01,
          -1.4713e-01,  4.1057e-01,  4.6748e-01, -2.7929e-01,  5.1891e-04,
           4.4669e-01, -6.0532e-01,  6.5746e-01, -7.1143e-01,  7.6275e-02],
         [-5.8744e-01,  5.7761e-03,  5.8907e-01,  3.2566e-01,  5.7920e-01,
           3.0107e-01, -5.3877e-01,  6.4468e-01, -5.3683e-01,  5.4830e-01,
          -2.4934e-01,  3.5482e-01,  4.6511e-01, -5.8961e-02, -5.7225e-02,
           4.3174e-01, -5.5028e-01,  6.5643e-01, -7.9412e-01,  2.4083e-01],
         [-6.3002e-01, -6.3183e-02,  6.2020e-01,  3.8596e-01,  6.2488e-01,
           3.1016e-01, -5.2417e-01,  6.9536e-01, -5.5455e-01,  5.8865e-01,
          -2.7821e-01,  3.3229e-01,  4.5781e-01,  8.9466e-03, -7.6020e-02,
           4.2058e-01, -5.3132e-01,  6.5376e-01, -8.1640e-01,  2.8635e-01],
         [-5.6746e-01, -8.4253e-02,  5.4777e-01,  3.5892e-01,  5.6911e-01,
           2.7855e-01,